## 🧤 Analysing Goalkeeper Performance – Women’s Euros 2025 & World Cup 2023

**Competitions:** 
**Purpose:** Analyse and compare goalkeeper performance using event-level data  
**Methods:** Event data analysis, rule-based classification of goalkeeper actions, exploratory statistics  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  

---

**Notebook first written:** `17/01/2026`  
**Last updated:** `18/01/2026`  

> This notebook focuses on analysing goalkeeper performance in elite women’s international football using detailed event data.  
>
> It examines goalkeeper involvement across matches, including saves, goals conceded, positive and negative interventions, and overall participation. Given the complexity of goalkeeper events in StatsBomb data, actions are classified using official goalkeeper type–outcome mappings to ensure consistency and interpretability.  
>
> The analysis provides aggregated metrics at goalkeeper and team level, such as participations, shots saved, goals conceded, and games played. This framework can be extended to incorporate minutes played, per-90 normalisation, or advanced shot-quality models (e.g. post-shot xG) for deeper evaluation.

In [1]:
from statsbombpy import sb
import pandas as pd
import numpy as np
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
import pprint
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Call statsbomb API to get all free competitions, then check Women's comps
free_comps = sb.competitions()

premier_league = free_comps[free_comps['competition_name'] == 'Premier League']
premier_league

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
64,2,27,England,Premier League,male,False,False,2015/2016,2025-04-23T14:36:29.347042,2021-06-13T16:17:31.694,None,2025-04-23T14:36:29.347042
65,2,44,England,Premier League,male,False,False,2003/2004,2025-06-24T13:53:07.585114,2021-06-13T16:17:31.694,None,2025-06-24T13:53:07.585114


In [7]:
free_comps.sort_values(by="season_name", ascending=False)


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
71,53,315,Europe,UEFA Women's Euro,female,False,True,2025,2025-07-28T14:19:20.467348,2025-07-29T16:03:07.355174,2025-07-29T16:03:07.355174,2025-07-28T14:19:20.467348
21,223,282,South America,Copa America,male,False,True,2024,2024-11-04T16:15:34.886972,None,None,2024-11-04T16:15:34.886972
68,55,282,Europe,UEFA Euro,male,False,True,2024,2024-09-28T16:51:20.698794,2025-03-24T14:12:30.785094,2025-03-24T14:12:30.785094,2024-09-28T16:51:20.698794
0,9,281,Germany,1. Bundesliga,male,False,False,2023/2024,2024-09-28T20:46:38.893391,2025-07-06T04:26:07.636270,2025-07-06T04:26:07.636270,2024-09-28T20:46:38.893391
2,1267,107,Africa,African Cup of Nations,male,False,True,2023,2024-09-28T01:57:35.846538,None,None,2024-09-28T01:57:35.846538
...,...,...,...,...,...,...,...,...,...,...,...,...
19,16,71,Europe,Champions League,male,False,False,1971/1972,2024-02-12T14:25:01.735880,2021-06-13T16:17:31.694,None,2024-02-12T14:25:01.735880
20,16,276,Europe,Champions League,male,False,False,1970/1971,2024-02-13T14:24:12.213582,None,None,2024-02-13T14:24:12.213582
34,43,272,International,FIFA World Cup,male,False,True,1970,2024-02-13T14:23:06.735299,None,None,2024-02-13T14:23:06.735299
35,43,270,International,FIFA World Cup,male,False,True,1962,2023-06-26T10:38:00.323984,None,None,2023-06-26T10:38:00.323984


In [58]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

matches_premier = sb.matches(competition_id=2, season_id=27)

matches_euro = sb.matches(competition_id=55, season_id=282)
matches_euro.head(5)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3942819,2024-07-10,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,England,1,2,available,available,2024-07-15T07:54:23.583297,2024-07-15T07:57:36.684453,6,Semi-finals,Signal-Iduna-Park,Felix Zwayer,Ronald Koeman,Gareth Southgate,1.1.0,2,2
1,3943043,2024-07-14,22:00:00.000,Europe - UEFA Euro,2024,Spain,England,2,1,available,available,2024-07-15T15:48:50.315500,2024-07-15T15:52:24.778809,7,Final,Olympiastadion Berlin,François Letexier,Luis de la Fuente Castillo,Gareth Southgate,1.1.0,2,2
2,3942752,2024-07-09,22:00:00.000,Europe - UEFA Euro,2024,Spain,France,2,1,available,available,2024-07-10T08:51:07.032303,2024-07-10T13:01:26.597509,6,Semi-finals,Allianz Arena,Slavko Vinčić,Luis de la Fuente Castillo,Didier Deschamps,1.1.0,2,2
3,3942382,2024-07-06,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,Turkey,2,1,available,available,2024-07-10T06:50:04.068615,2024-07-10T06:56:35.146850,5,Quarter-finals,Olympiastadion Berlin,Clément Turpin,Ronald Koeman,Vincenzo Montella,1.1.0,2,2
4,3942349,2024-07-05,22:00:00.000,Europe - UEFA Euro,2024,Portugal,France,0,0,available,available,2024-07-12T01:57:15.210934,2024-07-12T02:00:16.871522,5,Quarter-finals,Volksparkstadion,Michael Oliver,Roberto Martínez Montoliú,Didier Deschamps,1.1.0,2,2


In [59]:
events_df = pd.concat(
    [
        sb.events(match_id=mid)
        for mid in matches_euro["match_id"]
    ],
    ignore_index=True
)

In [60]:
events_df.tail()

,50_50,bad_behaviour_card,ball_receipt_outcome,block_deflection,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,counterpress,dribble_outcome,duel_outcome,duel_type,duration,foul_committed_advantage,foul_committed_card,foul_committed_penalty,foul_won_advantage,foul_won_defensive,foul_won_penalty,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_technique,goalkeeper_type,id,index,injury_stoppage_in_chain,interception_outcome,location,match_id,minute,off_camera,out,pass_aerial_won,pass_angle,pass_assisted_shot_id,pass_body_part,pass_cross,pass_cut_back,pass_deflected,pass_end_location,pass_goal_assist,pass_height,pass_length,pass_outcome,pass_outswinging,pass_recipient,pass_recipient_id,pass_shot_assist,pass_switch,pass_technique,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_outcome,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure,ball_recovery_recovery_failure,block_offensive,dribble_nutmeg,dribble_overrun,foul_committed_offensive,pass_inswinging,pass_no_touch,shot_one_on_one,clearance_other,foul_committed_type,miscontrol_aerial_won,goalkeeper_shot_saved_to_post,pass_straight,shot_open_goal,shot_saved_to_post,goalkeeper_success_in_play,ball_recovery_offensive,dribble_no_touch,pass_miscommunication,player_off_permanent,goalkeeper_penalty_saved_to_post,goalkeeper_shot_saved_off_target,shot_follows_dribble,shot_saved_off_target,shot_redirect
187853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9978110b-4c90-4bac-9697-3085af7c146f,2913,NaN,NaN,NaN,3930158,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,From Corner,Toni Kroos,5574.0,Left Defensive Midfield,121,Germany,770,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tactical,103.0,Emre Can,3493.0,NaN,Germany,770,00:35:02.486,Substitution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40b3fe92-926c-461f-964d-65ed52733b30,2925,NaN,NaN,NaN,3930158,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,Regular Play,Ryan Christie,12510.0,Center Forward,123,Germany,770,NaN,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tactical,103.0,Lawrence Shankland,38169.0,NaN,Scotland,942,00:35:54.594,Substitution,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,356d8da8-aafe-4e77-8b13-932d55573ee4,2558,NaN,NaN,"[15.1, 46.4]",3930158,67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,From Throw In,Grant Campbell Hanley,9481.0,Center Back,107,Germany,770,NaN,41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Scotland,942,00:22:41.537,Error,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,deb84edb-b9fb-45a3-98e1-a6b1d256ea03,3100,NaN,NaN,"[109.8, 47.3]",3930158,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,From Free Kick,NaN,NaN,NaN,132,Scotland,942,[5f131b45-721a-4555-bc1b-a3dcf7f6ca8c],32,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [61]:
goalkeeper = events_df[
    events_df["type"] == "Goal Keeper"
]

In [62]:
goalkeeper = goalkeeper.dropna(axis=1, how="all")
goalkeeper.tail(4)

,duration,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_technique,goalkeeper_type,id,index,location,match_id,minute,off_camera,out,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,team,team_id,timestamp,type,under_pressure,goalkeeper_shot_saved_to_post,goalkeeper_success_in_play,goalkeeper_penalty_saved_to_post,goalkeeper_shot_saved_off_target
187822,0.0,Both Hands,NaN,Success,Set,NaN,Standing,Shot Saved,30f66378-c801-4e19-b6f0-8efc40237628,3008,"[1.8, 38.8]",3930158,82,NaN,NaN,2,Regular Play,Angus Gunn,22379.0,Goalkeeper,126,Scotland,942,[defe8eb9-a518-4385-aca4-b33556d27463],54,Scotland,942,00:37:54.380,Goal Keeper,NaN,NaN,NaN,NaN,NaN
187823,0.0,NaN,"[4.1, 40.3]",NaN,Set,NaN,NaN,Shot Faced,9c3bebf5-1f99-49ee-b807-b21a2871661b,3084,"[4.1, 40.3]",3930158,85,NaN,NaN,2,Regular Play,Angus Gunn,22379.0,Goalkeeper,130,Germany,770,[39107e2c-b6b5-490b-a5e1-b55296af1202],12,Scotland,942,00:40:12.585,Goal Keeper,NaN,NaN,NaN,NaN,NaN
187824,0.0,NaN,"[1.1, 37.1]",NaN,Set,NaN,NaN,Shot Faced,da379529-5036-43bd-aeec-7fe3c7dbe96b,3099,"[1.1, 37.1]",3930158,86,NaN,NaN,2,From Free Kick,Manuel Neuer,5570.0,Goalkeeper,132,Scotland,942,[1759ff23-09e4-4170-8a38-90f97cea79e9],31,Germany,770,00:41:31.616,Goal Keeper,NaN,NaN,NaN,NaN,NaN
187825,0.0,Left Hand,NaN,Touched In,Set,NaN,Diving,Goal Conceded,38454a4e-9146-4ea9-a030-a218f89577ec,3371,"[0.5, 39.9]",3930158,92,NaN,NaN,2,Regular Play,Angus Gunn,22379.0,Goalkeeper,138,Scotland,942,[6ed1aa99-3539-41d1-9dcd-27badaa5e837],33,Scotland,942,00:47:33.521,Goal Keeper,NaN,NaN,NaN,NaN,NaN


In [63]:
(
    goalkeeper
    .groupby(["goalkeeper_type", "goalkeeper_outcome"])
    .size()
    .reset_index(name="count")
    .sort_values("goalkeeper_type", ascending=True)
)


,goalkeeper_type,goalkeeper_outcome,count
0,Collected,Collected Twice,8
1,Collected,Fail,5
2,Collected,Success,59
3,Goal Conceded,No Touch,87
4,Goal Conceded,Touched In,11
5,Keeper Sweeper,Claim,101
6,Keeper Sweeper,Clear,24
7,Penalty Conceded,No Touch,28
8,Penalty Saved,In Play Danger,1
9,Penalty Saved,In Play Safe,5


Define positive / negative outcomes (from the guide)

✅ POSITIVE interventions  
In general:
+ Any successful save / claim / clearance
+ Outcomes indicating control or safety

❌ NEGATIVE interventions  
Anything indicating:
+ goal conceded
+ failure
+ danger created

In [68]:
POSITIVE_OUTCOMES = {
    "Success",
    "Won",
    "Claim",
    "Clear",
    "In Play Safe",
    "Saved Twice",
    "Touched Out",
    "Collected Twice"
}

NEGATIVE_OUTCOMES = {
    "Fail",
    "Lost",
    "In Play Danger",
    "No Touch",
    "Touched In"
}

In [69]:
goalkeeper["is_positive"] = goalkeeper["goalkeeper_outcome"].isin(POSITIVE_OUTCOMES)
goalkeeper["is_negative"] = goalkeeper["goalkeeper_outcome"].isin(NEGATIVE_OUTCOMES)

In [77]:
goalkeeper_summary = (
    goalkeeper
    .groupby(["player", "team"])
    .agg(
        participations=("goalkeeper_outcome", "count"),
        positive_interventions=("is_positive", "sum"),
        negative_interventions=("is_negative", "sum"),
        games_played=("match_id", "nunique")
    )
    .reset_index()
    .sort_values("participations", ascending=False)
)


In [82]:
goalkeeper_summary["net_interventions"] = (
    goalkeeper_summary["positive_interventions"]
    - goalkeeper_summary["negative_interventions"]
)

goalkeeper_summary["positive_percentage"] = (
    round(goalkeeper_summary["positive_interventions"]
    / goalkeeper_summary["participations"] ,3)
)

In [86]:
goalkeeper_summary = goalkeeper_summary.sort_values(
    "positive_percentage",
    ascending=False
)

goalkeeper_summary

,player,team,participations,positive_interventions,negative_interventions,games_played,net_interventions,positive_percentage
16,Koen Casteels,Belgium,23,20,3,4,17,0.870
22,Predrag Rajković,Serbia,20,17,3,3,14,0.850
8,Fehmi Mert Günok,Turkey,33,27,6,4,21,0.818
9,Florin Constantin Niţă,Romania,36,28,8,4,20,0.778
19,Matěj Kovář,Czech Republic,4,3,1,1,2,0.750
15,Kasper Schmeichel,Denmark,20,15,5,4,10,0.750
5,David Raya Martin,Spain,4,3,1,1,2,0.750
18,Martin Dúbravka,Slovakia,39,29,10,4,19,0.744
25,Unai Simón Mendibil,Spain,35,26,9,6,17,0.743
3,Angus Gunn,Scotland,34,25,9,3,16,0.735


In [87]:
# Goal conceded (including penalties)
GOAL_CONCEDED_TYPES = {
    "Goal Conceded",
    "Penalty Conceded"
}

# Shot saves (ONLY real saves from shots)
SHOT_SAVE_TYPES = {
    "Shot Saved",
    "Shot Saved Off T",
    "Shot Saved To Post",
    "Penalty Saved",
    "Penalty Saved To Post"
}

SHOT_SAVE_OUTCOMES = {
    "Success",
    "In Play Danger",
    "In Play Safe",
    "Saved Twice",
    "Touched Out"
}

goalkeeper["is_goal_conceded"] = goalkeeper["goalkeeper_type"].isin(
    GOAL_CONCEDED_TYPES
)

goalkeeper["is_shot_saved"] = (
    goalkeeper["goalkeeper_type"].isin(SHOT_SAVE_TYPES)
    & goalkeeper["goalkeeper_outcome"].isin(SHOT_SAVE_OUTCOMES)
)

In [88]:
goalkeeper_summary = (
    goalkeeper
    .groupby(["player", "team"])
    .agg(
        participations=("goalkeeper_type", "count"),
        positive_interventions=("is_positive", "sum"),
        negative_interventions=("is_negative", "sum"),
        shots_saved=("is_shot_saved", "sum"),
        goals_conceded=("is_goal_conceded", "sum"),
        games_played=("match_id", "nunique")
    )
    .reset_index()
    .sort_values("positive_interventions", ascending=False)
)

In [89]:
goalkeeper_summary

,player,team,participations,positive_interventions,negative_interventions,shots_saved,goals_conceded,games_played
11,Giorgi Mamardashvili,Georgia,119,33,14,28,8,4
14,Jordan Pickford,England,91,30,11,19,9,7
18,Martin Dúbravka,Slovakia,76,29,10,12,5,4
9,Florin Constantin Niţă,Romania,85,28,8,15,6,4
20,Mike Maignan,France,72,28,11,15,6,6
8,Fehmi Mert Günok,Turkey,72,27,6,15,4,4
25,Unai Simón Mendibil,Spain,83,26,9,14,3,6
3,Angus Gunn,Scotland,59,25,9,13,7,3
4,Bart Verbruggen,Netherlands,77,21,14,18,6,6
12,Jan Oblak,Slovenia,77,20,10,14,5,4


## Related events

In [64]:
goalkeeper_rel = (
    goalkeeper
    .explode("related_events")
    .merge(
        events_df,
        how="left",
        left_on="related_events",
        right_on="id",
        suffixes=("", "_right")
    )
    .rename(columns={
        f"{col}_right": f"related_{col}"
        for col in events_df.columns
        if col != "id"
    })
    .drop(columns=["id"], errors="ignore")
    .dropna(axis=1, how="all")
)


In [65]:
goalkeeper_rel.head()

,duration,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_technique,goalkeeper_type,index,location,match_id,minute,off_camera,out,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,team,team_id,timestamp,type,under_pressure,goalkeeper_shot_saved_to_post,goalkeeper_success_in_play,goalkeeper_penalty_saved_to_post,goalkeeper_shot_saved_off_target,dribble_outcome,related_duration,id_right,related_index,related_location,related_match_id,related_minute,related_off_camera,related_out,pass_aerial_won,pass_angle,pass_body_part,pass_cross,pass_end_location,pass_height,pass_length,pass_outcome,pass_outswinging,pass_recipient,pass_recipient_id,pass_switch,pass_technique,pass_through_ball,pass_type,related_period,related_play_pattern,related_player,related_player_id,related_position,related_possession,related_possession_team,related_possession_team_id,related_related_events,related_second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_outcome,shot_statsbomb_xg,shot_technique,shot_type,related_team,related_team_id,related_timestamp,related_type,related_under_pressure,pass_inswinging,shot_one_on_one,shot_open_goal,shot_saved_to_post,shot_follows_dribble,shot_saved_off_target,shot_redirect
0,0.0,NaN,NaN,Claim,NaN,NaN,NaN,Keeper Sweeper,198,"[8.2, 33.1]",3942819,3,NaN,NaN,1,From Keeper,Jordan Pickford,3468.0,Goalkeeper,8,England,768,9e069da0-4b6e-45c2-aaa9-5f4c6e60db81,37,England,768,00:03:37.728,Goal Keeper,True,NaN,NaN,NaN,NaN,NaN,2.829870,9e069da0-4b6e-45c2-aaa9-5f4c6e60db81,194.0,"[83.9, 60.8]",3942819.0,3.0,NaN,NaN,NaN,-0.457917,Right Foot,NaN,"[111.9, 47.0]",Ground Pass,31.21602,Incomplete,NaN,Donyell Malen,15582.0,NaN,NaN,NaN,NaN,1.0,Regular Play,Xavi Simons,39167.0,Center Attacking Midfield,7.0,Netherlands,941.0,"[9ffe3a7a-63f2-4373-bc79-155ab0e9ca75, b70fe9a...",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Netherlands,941.0,00:03:34.898,Pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,Claim,NaN,NaN,NaN,Keeper Sweeper,198,"[8.2, 33.1]",3942819,3,NaN,NaN,1,From Keeper,Jordan Pickford,3468.0,Goalkeeper,8,England,768,9ef92f0a-9371-4254-b061-b81b84128b34,37,England,768,00:03:37.728,Goal Keeper,True,NaN,NaN,NaN,NaN,NaN,0.842624,9ef92f0a-9371-4254-b061-b81b84128b34,196.0,"[106.9, 48.2]",3942819.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Regular Play,Donyell Malen,15582.0,Right Wing,7.0,Netherlands,941.0,"[1fdb1372-63d4-4a38-9c08-b7682ccc0817, b70fe9a...",37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Netherlands,941.0,00:03:37.018,Pressure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,Right Hand,NaN,Touched In,Set,NaN,Standing,Goal Conceded,343,"[2.7, 38.6]",3942819,6,NaN,NaN,1,Regular Play,Jordan Pickford,3468.0,Goalkeeper,11,Netherlands,941,e0159fb4-47e8-422e-9216-6e44cf173408,42,England,768,00:06:42.310,Goal Keeper,NaN,NaN,NaN,NaN,NaN,NaN,0.865482,e0159fb4-47e8-422e-9216-6e44cf173408,342.0,"[99.6, 51.2]",3942819.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Regular Play,Xavi Simons,39167.0,Center Attacking Midfield,11.0,Netherlands,941.0,[5820106f-bcbd-4d37-afe0-88d3cf9baf73],41.0,NaN,Right Foot,NaN,"[120.0, 37.4, 2.0]",NaN,"[{'location': [99.2, 65.4], 'player': {'id': 1...",NaN,Goal,0.048935,Normal,Open Play,Netherlands,941.0,00:06:41.445,Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Both Hands,NaN,Touched Out,Set,NaN,Standing,Shot Saved,447,"[1.7, 40.4]",3942819,12,NaN,NaN,1,Regular Play,Bart Verbruggen,37274.0,Goalkeeper,17,England,768,0ec0210f-4dfb-4a1d-ab52-4c1f2156b219,40,Netherlands,941,00:12:40.707,Goal Keeper,NaN,NaN,NaN,NaN,NaN,NaN,0.935472,0ec0210f-4dfb-4a1d-ab52-4c1f2156b219,446.0,"[88.2, 36.6]",3942819.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Regular Play,Harry Kane,10955.0,Center Forward,17.0,England,768.0,[fd31ed99-e284-